# Bond/Futures quotation converter


In [26]:
def quotation(quote, conversion_type):
    import re

    # Convert futures quotes to normal quotes    
    if conversion_type == 0:  
        integer_part, fraction_part = re.split(r"[-']", quote)
        integer_part = int(integer_part)

        # Extract the first two digits and the trailing digit(s) of the fraction part
        if len(fraction_part) == 2:
            first_two_digits = int(fraction_part)
            decimal_price = integer_part + (first_two_digits / 32)
        elif len(fraction_part) == 3:
            first_two_digits = int(fraction_part[:-1])
            trailing_digit = fraction_part[-1]  # Modify this line

            # Apply the pricing rule based on the trailing digit
            if trailing_digit == '0':
                decimal_price = integer_part + (first_two_digits / 32)
            elif trailing_digit == '2':
                decimal_price = integer_part + (first_two_digits / 32) + (0.25 * (1 / 32))
            elif trailing_digit == '5' or trailing_digit == '+':
                decimal_price = integer_part + (first_two_digits / 32) + (0.5 * (1 / 32))
            elif trailing_digit == '6' or trailing_digit == '7':
                decimal_price = integer_part + (first_two_digits / 32) + (0.75 * (1 / 32))
            else:
                raise ValueError("Invalid trailing digit")
        else:
            raise ValueError("Invalid fraction part length")

        return decimal_price
    
    # Convert normal quotes to future quotes
    elif conversion_type == 1:  # Convert decimal to quote
        decimal_price = float(quote)
        integer_part = int(decimal_price)
        fractional_part = decimal_price - integer_part

        # Calculate the 32nds and the remainder
        fractional_part_32nds = int(fractional_part * 32)
        remainder = (fractional_part * 32) - fractional_part_32nds

        # Check if the remainder is close enough to the whole number
        if 0.9 <= remainder < 1:
            fractional_part_32nds += 1
            remainder -= 1

        # Check if the fractional part is one of the special cases (0.25, 0.5, or 0.75)
        if remainder in [0.25, 0.5, 0.75]:
            trailing_digit = ['2', '5', '7'][int(remainder * 4) - 1]
            futures_quote = f"{integer_part}-{fractional_part_32nds:02}{trailing_digit}"
        else:
            futures_quote = f"{integer_part}-{fractional_part_32nds:02}"

        return futures_quote

    else:
        raise ValueError("Invalid conversion type. Use 0 for quote to decimal or 1 for decimal to quote.")

In [30]:
# Convert quote to decimal
quote = "105-16+"
decimal_price = quotation(quote, 0)
print(f"Decimal price of {quote} = {decimal_price}")

# Convert decimal to quote
decimal_price = 105.515625
quote = quotation(decimal_price, 1)
print(f"Futures price of {decimal_price} = {quote}")

Decimal price of 105-16+ = 105.515625
Futures price of 105.515625 = 105-165
